In [1]:
#!pip install ipympl matplotlib numpy

In [2]:
# from google.colab import output
# output.enable_custom_widget_manager()

In [3]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from math import *
from itertools import product
from IPython.display import clear_output

In [4]:
with plt.ioff():
    fig, ax = plt.subplots()

In [5]:
from ipywidgets import AppLayout
from ipywidgets import Dropdown, FloatText, IntText, Button, Layout, HTML, HBox, VBox

In [6]:
rng = np.random.default_rng(3123)
functions = {
    "ax^3+bx^2+cx+d":
    (np.vectorize(lambda x, a, b, c, d: a*x**3 + b*x**2 + c*x + d), {"a": [-3, 3], "b": [-3, 3], "c": [-3, 3], "d": [-3, 3]}),
    "x*sin(2*pi*x)":
    (np.vectorize(lambda x: x*sin(2*pi*x)), {}),
}

In [7]:
epsilon_rngs = {
    "normal":
    (lambda n, sigma, center=0: rng.normal(0, sigma, n), {"center": None, "sigma": None}),
    "uniform":
    (lambda n, ep0: rng.uniform(-ep0, ep0, n), {"ep0": None}),
}

In [8]:
def update_plot(f_name, f, f_params, N, e_rng, e_rng_name, e_params):
    global ax
    epsilon = e_rng(N, **e_params)
    x = np.linspace(-1, 1, 100)
    ax.clear()
    ax.plot(x, f(x, **f_params), "b", label=f_name)
    x_c = rng.uniform(-1, 1, N)
    y_c = f(x_c, **f_params) + epsilon
    ax.scatter(x_c, y_c, c="r")
    ax.legend()
    ax.title.set_text(f"N: {N} ep_rng: {e_rng_name}")

In [9]:
def linear_regression():
    global ax, app
    data = ax.collections[0].get_offsets()
    N = data.shape[0]
    x_ep = data.data[:, 0]
    y_ep = data.data[:, 1]
    x, y = ax.lines[0].get_data()

    A = np.vander(x_ep, N)
    w = np.linalg.solve(A, y_ep)[::-1]
    ft = np.vectorize(lambda x: np.sum(np.power(x, np.arange(N)) * w))
    Er = np.sum((ft(x) - y)**2) / 2
    clear_output(wait=True)

    ax.plot(x, ft(x), "g", label=f"f' - {Er}")

    ax.axis([min(x) - 1, max(x) + 1, min(y) - 2, max(y) + 2])
    ax.legend()
    

In [10]:
def create_param_inputs(functions: dict[str, tuple[callable, dict[str, list[float]]]]):

    function_blocks = dict()
    function_params_inputs = dict()

    for f_name, (_, params) in functions.items():

        function_box_items = list()
        function_inputs = list()

        for param_name, param_range in params.items():
            rng_btn = None
            if param_range is None:
                param_range = [0, 1]
            else:
                rng_btn = Button()
            inp = FloatText(
                value = rng.uniform(*param_range),
                step = .1,
                description=param_name,
                layout=Layout(width="95%")
            )
            function_inputs.append(inp)
            box_items = list()
            box_items.append(inp)

            if rng_btn is not None:
                rng_btn.linked_input = inp
                rng_btn.linked_input_range = param_range
                # rng_btn.on_click(lambda x: inp.__setattr__("value", rng.uniform(*param_range)))
                rng_btn.on_click(lambda x: x.linked_input.__setattr__("value", rng.uniform(*x.linked_input_range)))
                box_items.append(rng_btn)
            
            box = HBox(
                box_items
            )
            function_box_items.append(box)
        
        function_blocks[f_name] = VBox(
            function_box_items,
            layout=Layout(width="100%", display="none")
        )
        function_params_inputs[f_name] = function_inputs
    
    return function_blocks, function_params_inputs            


In [12]:
first_rng = next(iter(epsilon_rngs.keys()))
ep_rng_dropdown = Dropdown(
    options=epsilon_rngs.keys(),
    value=first_rng,
    layout=Layout(width="95%"),
    description="ε_rng",
    style={"description_width": "initial"},
)

ep_rng_blocks, ep_rng_params = create_param_inputs(epsilon_rngs)
# ep_rng_params = {
#     key: VBox(
#         [FloatText(value=0, step=.1, description=param, layout=Layout(width="95%")) for param in params],
#         layout=Layout(width="100%", display="none"),
#         ) for key, (_, params) in epsilon_rngs.items()
# }

def on_dropdown_change(value, boxes: dict[str, VBox]):
    old_box = boxes[value["old"]]
    new_box = boxes[value["new"]]
    old_box.layout.display="none"
    new_box.layout.display="block"

ep_rng_dropdown.observe(lambda value: on_dropdown_change(value, ep_rng_blocks), names="value")
ep_rng_blocks[first_rng].layout.display = "block"

first_function = next(iter(functions.keys()))
function_dropdown = Dropdown(
    options=functions.keys(),
    value=first_function,
    layout=Layout(width="100%"),
    description="function",
    style={"description_width": "initial"},
)
functions_blocks, functions_params = create_param_inputs(functions)
# function_params = {
#     key: VBox(
#         [FloatText(value=0, step=.1, description=param, layout=Layout(width="95%")) for param in params],
#         layout=Layout(width="100%", display="none"),
#         ) for key, (_, params) in functions.items()
# }
function_dropdown.observe(lambda value: on_dropdown_change(value, functions_blocks), names="value")
functions_blocks[first_function].layout.display="block"

n_input = IntText(
                value=20,
                min=1,
                max=100,
                step=1,
                layout=Layout(width="95%"),
                description="N",
                style={"description_width": "initial"},
)
AppLayout(
    center=fig.canvas,
    header=VBox(
        [
            update_bt := Button(
                description="generate points",
                layout=Layout(width="98%"),
                style={"description_width": "initial"},
            ),
            linear_regression_bt := Button(
                description="draw linear regression",
                layout=Layout(width="98%"),
                style={"description_width": "initial"},
            )
        ]
    ),
    footer=VBox(
        [
            function_dropdown,
            *functions_blocks.values()
        ]
    ),
    right_sidebar=VBox(
        [
            n_input,
            ep_rng_dropdown,
            *ep_rng_blocks.values()
        ],
        layout=Layout(height="100%", width="95%")
    ),
    pane_heights=["80px", 3, 1],
)

AppLayout(children=(VBox(children=(Button(description='generate points', layout=Layout(width='98%'), style=But…

In [ ]:
update_bt.on_click(lambda x: update_plot(
    function_dropdown.value,
    functions[function_dropdown.value][0],
    {w.description: w.value for w in functions_params[function_dropdown.value]},
    n_input.value,
    epsilon_rngs[ep_rng_dropdown.value][0],
    ep_rng_dropdown.value,
    {w.description: w.value for w in ep_rng_params[ep_rng_dropdown.value]},
))
linear_regression_bt.on_click(lambda x: linear_regression())